# # Análisis de Augmentación de Audio para Emergencias
# 
# Este notebook analiza los efectos de la augmentación en los datos de audio para entrenamiento de ASR en contextos de emergencia.



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
import soundfile as sf
from pathlib import Path
import IPython.display as ipd
from src.utils.config_loader import load_config

In [ ]:
# Configuración
config = load_config("augment_config")
languages = load_config("languages")["languages"]

In [ ]:
# Cargar metadata aumentada
try:
    df_augmented = pd.read_csv("data/generated/metadata.csv")
    print(f"Dataset aumentado: {len(df_augmented)} muestras")
    print("\nDistribución por nivel de ruido:")
    print(df_augmented['noise_level'].value_counts())
except FileNotFoundError:
    print("Primero ejecuta el script de augmentación")
    df_augmented = pd.DataFrame()

In [ ]:
# Visualizar distribución de características
if not df_augmented.empty:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Distribución por idioma
    df_augmented['language'].value_counts().plot(kind='bar', ax=axes[0,0], title='Distribución por Idioma')
    
    # Distribución por nivel de ruido
    df_augmented['noise_level'].value_counts().plot(kind='bar', ax=axes[0,1], title='Distribución por Nivel de Ruido')
    
    # Distribución por escenario
    if 'scenario' in df_augmented.columns:
        df_augmented['scenario'].value_counts().plot(kind='bar', ax=axes[1,0], title='Distribución por Escenario')
    
    # Distribución de SNR
    if 'snr' in df_augmented.columns:
        df_augmented['snr'].hist(bins=30, ax=axes[1,1], alpha=0.7)
        axes[1,1].set_title('Distribución de SNR (dB)')
        axes[1,1].set_xlabel('SNR (dB)')
        axes[1,1].set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Análisis de efectos aplicados
if not df_augmented.empty and 'effect' in df_augmented.columns:
    effects_counts = df_augmented['effect'].value_counts()
    plt.figure(figsize=(12, 6))
    effects_counts.head(10).plot(kind='bar')
    plt.title('Efectos de Audio Más Comunes')
    plt.xlabel('Tipo de Efecto')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualización de waveforms y espectrogramas
def plot_audio_comparison(clean_path, augmented_path, title1="Original", title2="Aumentado"):
    """Comparar audio original vs aumentado"""
    # Cargar audios
    y_clean, sr = librosa.load(clean_path, sr=16000)
    y_aug, sr_aug = librosa.load(augmented_path, sr=16000)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 8))
    
    # Waveforms
    librosa.display.waveshow(y_clean, sr=sr, ax=axes[0,0], alpha=0.7)
    axes[0,0].set_title(f'{title1} - Waveform')
    axes[0,0].set_ylabel('Amplitud')
    
    librosa.display.waveshow(y_aug, sr=sr_aug, ax=axes[0,1], alpha=0.7, color='orange')
    axes[0,1].set_title(f'{title2} - Waveform')
    axes[0,1].set_ylabel('Amplitud')
    
    # Espectrogramas
    S_clean = librosa.feature.melspectrogram(y=y_clean, sr=sr, n_mels=128)
    S_db_clean = librosa.power_to_db(S_clean, ref=np.max)
    librosa.display.specshow(S_db_clean, sr=sr, x_axis='time', y_axis='mel', ax=axes[1,0])
    axes[1,0].set_title(f'{title1} - Espectrograma')
    axes[1,0].set_ylabel('Frecuencia (Hz)')
    
    S_aug = librosa.feature.melspectrogram(y=y_aug, sr=sr_aug, n_mels=128)
    S_db_aug = librosa.power_to_db(S_aug, ref=np.max)
    librosa.display.specshow(S_db_aug, sr=sr_aug, x_axis='time', y_axis='mel', ax=axes[1,1])
    axes[1,1].set_title(f'{title2} - Espectrograma')
    axes[1,1].set_ylabel('Frecuencia (Hz)')
    
    plt.colorbar(axes[1,0].images[0], ax=axes[1,0])
    plt.colorbar(axes[1,1].images[0], ax=axes[1,1])
    plt.tight_layout()
    plt.show()
    # Reproducir audios
    print(f"Audio {title1}:")
    ipd.display(ipd.Audio(clean_path))
    print(f"Audio {title2}:")
    ipd.display(ipd.Audio(augmented_path))

In [ ]:
# Ejemplo de comparación (si hay datos)
if not df_augmented.empty:
    # Encontrar un ejemplo de cada nivel de ruido
    examples = {}
    for level in ['clean', 'low_noise', 'medium_noise', 'high_noise']:
        level_samples = df_augmented[df_augmented['noise_level'] == level]
        if len(level_samples) > 0:
            examples[level] = level_samples.iloc[0]
    
    # Comparar clean vs aumentado
    if 'clean' in examples and 'medium_noise' in examples:
        clean_sample = examples['clean']
        augmented_sample = examples['medium_noise']
        
        print(f"Comparando: {clean_sample['audio_ref']} (clean) vs {augmented_sample['audio_ref']} (medium_noise)")
        plot_audio_comparison(
            clean_sample['path'], 
            augmented_sample['path'],
            "Clean", 
            f"Medium Noise (SNR: {augmented_sample.get('snr', 'N/A')}dB)"
        )

In [ ]:
# Análisis de características acústicas por nivel de ruido
def analyze_acoustic_features(df):
    """Analizar características acústicas por nivel de ruido"""
    features_data = []
    
    for _, row in df.iterrows():
        try:
            audio, sr = librosa.load(row['path'], sr=16000)
            
            # Calcular características
            rms = np.sqrt(np.mean(audio**2))
            zcr = np.mean(librosa.feature.zero_crossing_rate(audio))
            spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sr))
            
            features_data.append({
                'path': row['path'],
                'noise_level': row.get('noise_level', 'unknown'),
                'rms': rms,
                'zcr': zcr,
                'spectral_centroid': spectral_centroid,
                'duration': len(audio) / sr
            })
        except Exception as e:
            print(f"Error procesando {row['path']}: {e}")
            continue
    
    return pd.DataFrame(features_data)


In [ ]:
# Ejecutar análisis de características
if not df_augmented.empty:
    features_df = analyze_acoustic_features(df_augmented.sample(100))  # Muestra para velocidad
    
    # Visualizar características
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # RMS por nivel de ruido
    sns.boxplot(data=features_df, x='noise_level', y='rms', ax=axes[0,0])
    axes[0,0].set_title('RMS por Nivel de Ruido')
    axes[0,0].set_ylabel('RMS')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Zero-crossing rate por nivel de ruido
    sns.boxplot(data=features_df, x='noise_level', y='zcr', ax=axes[0,1])
    axes[0,1].set_title('Zero-Crossing Rate por Nivel de Ruido')
    axes[0,1].set_ylabel('ZCR')
    axes[0,1].tick_params(axis='x', rotation=45)
    
    # Spectral centroid por nivel de ruido
    sns.boxplot(data=features_df, x='noise_level', y='spectral_centroid', ax=axes[1,0])
    axes[1,0].set_title('Spectral Centroid por Nivel de Ruido')
    axes[1,0].set_ylabel('Spectral Centroid (Hz)')
    axes[1,0].tick_params(axis='x', rotation=45)
    
    # Duración por nivel de ruido
    sns.boxplot(data=features_df, x='noise_level', y='duration', ax=axes[1,1])
    axes[1,1].set_title('Duración por Nivel de Ruido')
    axes[1,1].set_ylabel('Duración (s)')
    axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Análisis de tipos de ruido más efectivos
if not df_augmented.empty and 'noise_type' in df_augmented.columns:
    noise_analysis = df_augmented.groupby('noise_type').agg({
        'path': 'count',
        'snr': 'mean'
    }).rename(columns={'path': 'count', 'snr': 'avg_snr'}).sort_values('count', ascending=False)
    
    print("Tipos de ruido más utilizados:")
    print(noise_analysis.head(10))
    
    # Visualizar
    plt.figure(figsize=(12, 6))
    noise_analysis.head(10)['count'].plot(kind='bar')
    plt.title('Tipos de Ruido Más Comunes')
    plt.xlabel('Tipo de Ruido')
    plt.ylabel('Frecuencia')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# Generar reporte de calidad de augmentación
def generate_augmentation_report(df):
    """Generar reporte de calidad de la augmentación"""
    report = {
        'total_samples': len(df),
        'languages_represented': df['language'].nunique(),
        'noise_levels_represented': df['noise_level'].nunique() if 'noise_level' in df.columns else 0,
        'scenarios_represented': df['scenario'].nunique() if 'scenario' in df.columns else 0,
        'avg_samples_per_language': len(df) / df['language'].nunique(),
        'data_balance_score': df['language'].value_counts().std() / df['language'].value_counts().mean()
    }
    
    return report

if not df_augmented.empty:
    aug_report = generate_augmentation_report(df_augmented)
    print("Reporte de Augmentación:")
    for key, value in aug_report.items():
        print(f"  {key}: {value:.2f}")


In [ ]:
# Guardar análisis
if not df_augmented.empty:
    analysis_output = {
        'augmentation_report': generate_augmentation_report(df_augmented),
        'language_distribution': df_augmented['language'].value_counts().to_dict(),
        'noise_level_distribution': df_augmented['noise_level'].value_counts().to_dict() if 'noise_level' in df_augmented.columns else {}
    }
    
    import json
    with open('augmentation_analysis.json', 'w', encoding='utf-8') as f:
        json.dump(analysis_output, f, indent=2, ensure_ascii=False)
    
    print("Análisis guardado en augmentation_analysis.json")